In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

from keras.applications.vgg16 import VGG16 # load VGG16 model from keras
from keras.preprocessing.image import load_img # Using images library
from keras.preprocessing.image import img_to_array # # convert image pixels to numpy for specific manipulations
from keras.applications.vgg16 import preprocess_input # to prepare for new input
from keras.applications.vgg16 import decode_predictions # for reporting probabilities

Using TensorFlow backend.
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [2]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# prepare an iterators for each dataset
train_it = train_datagen.flow_from_directory('data/images/train',
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     batch_size=32,
                                     class_mode='categorical',
                                     shuffle=True)
# val_it = datagen.flow_from_directory('data/validation/', class_mode='categorical')
test_it = train_datagen.flow_from_directory('data/images/test',
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     batch_size=32,
                                     class_mode='categorical',
                                     shuffle=True)
# confirm the iterator works
train_batchX, train_batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (train_batchX.shape, train_batchX.min(), train_batchX.max()))
test_batchX, test_batchy = test_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (test_batchX.shape, test_batchX.min(), test_batchX.max()))

Found 58 images belonging to 5 classes.
Found 16 images belonging to 5 classes.
Batch shape=(32, 224, 224, 3), min=-123.680, max=151.061
Batch shape=(16, 224, 224, 3), min=-123.680, max=151.061


In [3]:
# classifying only weapons 
vgg16_model = VGG16(weights="imagenet")

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation

seq_model = Sequential()

# iterate all layers in vgg16 to sequential for custom modification
for layer in vgg16_model.layers:
    seq_model.add(layer)
    
seq_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [5]:
# 1000 categories, pop off output layer to work only specific categories
seq_model._layers.pop()

seq_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [6]:
# freeze weights to prevent constantly updating weights
for layer in seq_model._layers:
    layer.trainable = False

In [7]:
seq_model.add(Dense(5, activation='softmax')) # 5 categories 

seq_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [8]:
seq_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train = train_it.n//train_it.batch_size
seq_model.fit_generator(generator=train_it,
                   steps_per_epoch=step_size_train,
                   epochs=10)


Epoch 1/10
1/1 [==============================] - 7s 7s/step - loss: 1.6042 - accuracy: 0.2308
Epoch 2/10
1/1 [==============================] - 7s 7s/step - loss: 1.5947 - accuracy: 0.3438
Epoch 3/10
1/1 [==============================] - 6s 6s/step - loss: 1.5833 - accuracy: 0.3846
Epoch 4/10
1/1 [==============================] - 7s 7s/step - loss: 1.5822 - accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 7s 7s/step - loss: 1.5731 - accuracy: 0.7188
Epoch 6/10
1/1 [==============================] - 6s 6s/step - loss: 1.5734 - accuracy: 0.6538
Epoch 7/10
1/1 [==============================] - 6s 6s/step - loss: 1.5635 - accuracy: 0.8077
Epoch 8/10
1/1 [==============================] - 7s 7s/step - loss: 1.5616 - accuracy: 0.7812
Epoch 9/10
1/1 [==============================] - 7s 7s/step - loss: 1.5592 - accuracy: 0.8125
Epoch 10/10
1/1 [==============================] - 6s 6s/step - loss: 1.5545 - accuracy: 0.7692


In [11]:
seq_model.predict(test_it)   

array([[0.1876    , 0.19248015, 0.20403124, 0.2176263 , 0.19826227],
       [0.19412397, 0.19418178, 0.20031889, 0.2130044 , 0.198371  ],
       [0.19267766, 0.20944346, 0.21485946, 0.19389355, 0.18912591],
       [0.19294594, 0.20906779, 0.21422367, 0.19446498, 0.18929769],
       [0.19825728, 0.1965793 , 0.19398919, 0.21102108, 0.20015308],
       [0.20151892, 0.18525778, 0.20120329, 0.20373893, 0.20828114],
       [0.19298331, 0.19507717, 0.20288016, 0.21589372, 0.19316572],
       [0.19665076, 0.19586083, 0.2022869 , 0.21590343, 0.18929814],
       [0.19506994, 0.19182189, 0.19920066, 0.21370423, 0.20020328],
       [0.20147146, 0.19378471, 0.19642428, 0.2083955 , 0.19992407],
       [0.20046802, 0.19550309, 0.20062917, 0.20494585, 0.19845393],
       [0.20051345, 0.1925588 , 0.19759725, 0.20693983, 0.20239063],
       [0.21276964, 0.19658022, 0.19087672, 0.18933041, 0.21044303],
       [0.20178033, 0.18452682, 0.20114471, 0.20331864, 0.2092295 ],
       [0.19655444, 0.19598548, 0.

In [12]:
pred = seq_model.predict(test_it)

# convert the probabilities to class labels
label = decode_predictions(pred)

# retrieve the most likely result, e.g. highest probability
label = label[0][0]

ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (16, 5)